In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 17 12:34:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


# 0 - Installs and Imports

In [4]:
!pip install docarray evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=46c930e16a3e1c9ed1a89e1d49ae173fcb3eba0d66dc4d2c4cff21a47c9bbe96
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [5]:
!pip install transformers

In [6]:
!pip install --upgrade langchain

In [7]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [8]:
import evaluate
import gradio as gr
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline, TrainingArguments, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, DataCollatorForLanguageModeling #DataCollatorForSeq2SeqLM
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain, RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown
import re
import numpy as np
import pandas as pd
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 1 - Setup

## 1.1 - Loading the Model

In [9]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model =  AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## 1.2 - Creating the Pipeline using LangChain

In [ ]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.3,
    return_full_text=False,
    pad_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=model_pipeline)

<ipython-input-9-50be81737a5f>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=model_pipeline)


## 1.3 - Template Prompt

In [ ]:
template_string = """Answer the following question in one short sentence. Do not ask follow-up questions.

Current conversation:
{history}
Human: {input}
AI Assistant:"""

prompt_template = PromptTemplate.from_template(
    template=template_string,
)

memory = ConversationBufferWindowMemory(k=1, memory_key="history", input_key="input")
llm_chain = ConversationChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True,
    memory=memory
)

# 2 - Using The Model

## 2.1 - Prompting The Model

In [ ]:
# prompt = prompt_template.format(input="How many legs does an elephant have?")
result = llm_chain.predict(input="How many legs does an elephant have?")
print(result)



> Entering new ConversationChain chain...
Prompt after formatting:
Answer the following question in one short sentence. Do not ask follow-up questions.

Current conversation:

Human: How many legs does an elephant have?
AI Assistant:

> Finished chain.
 Elephants have four limbs, and they are classified as mammals.


## 2.2 - Prompt Using Chat Interface

In [ ]:
def respond(message, chat_history):
    # Get response from conversation chain
    response = llm_chain.predict(input=message)

    # Update chat history
    chat_history.append((message, response))

    return "", chat_history

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

<ipython-input-22-9deb1b1357e6>:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=240) #just to fit the notebook


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e51ea99f76ae3c687.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 2.3 - RAG

In [ ]:
# import the product csv
file = 'products-100.csv'
loader = CSVLoader(file_path=file)

In [ ]:
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

<ipython-input-15-6c19354b71ea>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# create the vector database
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
docs = loader.load()
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)
retriever = db.as_retriever()

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
query = "Please list all product names that have Printer in their name"
response = qa_stuff.run(query)

<ipython-input-19-fdb08b45e6de>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

 The given text material contains multiple products with "Printer" as one or more of its parts. Some examples include compact printer air advanced digital, digital tablet router lte, clean printer advance premium sense, wireless tablet rter wi fi premium air, wirelessly connected printe rer, and wireless tablet rotr wrst kit

## 2.4 Fine-Tuning

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
# Load model and tokenizer again just in case
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [12]:
# Add special tokens if they don't exist
special_tokens = {"pad_token": "<PAD>"}
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens(special_tokens)

# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Set the pad_token_id in the model's configuration
model.config.pad_token_id = tokenizer.pad_token_id

In [13]:
# Creating the LoRA params for LoRA fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare model for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 6,307,840 || all params: 1,106,356,224 || trainable%: 0.5701


In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Loading the squad dataset
dataset = pd.read_json("drive/MyDrive/train.json", lines=True, nrows=20000)
validation = pd.read_json("drive/MyDrive/validation.json", lines=True,nrows=50)
dataset.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}"
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answer_start': [188]}"
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"{'text': ['the Main Building'], 'answer_start': [279]}"
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is the Grotto at Notre Dame?,"{'text': ['a Marian place of prayer and reflection'], 'answer_start': [381]}"
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of pra

In [15]:
def format_training_example(example):
    context = example["context"]
    question = example["question"]
    answer = example["answers"]["text"][0] if example["answers"]["text"] else "Unknown"

    input_text = f"""Below is a context and a question about it. Answer the question based on the given context.

    Context: {context}

    Question: {question}"""

    return {"input_text": input_text, "answer_text": answer + "</s>"}

In [16]:
rouge = evaluate.load("rouge")
def compute_metrics(pred):
    predictions = pred.predictions
    label_ids = pred.label_ids

    # If predictions are a tuple (e.g., when using generate), extract the first element
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # Ensure predictions and labels are NumPy arrays
    predictions = np.array(predictions)
    label_ids = np.array(label_ids)

    # Replace -100 in labels with the tokenizer's pad_token_id
    # This is necessary because -100 is used for ignoring loss but is not a valid token ID for decoding.
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Replace -100 in predictions with the tokenizer's pad_token_id if they exist
    # While predictions should ideally not have -100, adding this check makes the function more robust.
    predictions[predictions == -100] = tokenizer.pad_token_id


    # Decode predictions and labels
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # ROUGE expects references to be a list of list-of-strings
    decoded_labels = [[label] for label in decoded_labels]

    return rouge.compute(predictions=decoded_predictions, references=decoded_labels)

In [17]:
train_dataset = dataset.apply(format_training_example,axis=1)
validation_dataset = validation.apply(format_training_example,axis=1)

In [18]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["answer_text"], padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [19]:
tokenized_train_dataset = train_dataset.map(tokenize_function)
tokenized_validation_dataset = validation_dataset.map(tokenize_function)

In [20]:
# Setting the training arguments
output_dir = "./tiny-llama-squad-finetuned"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    eval_steps=200,
    predict_with_generate=True,
    generation_max_length=2048,
    save_steps=500,
    warmup_steps=100,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    # load_best_model_at_end=True,
    save_total_limit=3,
    fp16=device == "cuda",  # Use fp16 if on GPU
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [21]:
# Do the training
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # label_names=["answer_text"],
)

trainer.train()

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.083900,1.703402,0.428547,0.428249,0.427241,0.426157
2,0.581600,2.284380,0.493917,0.489437,0.492964,0.491156


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.083900,1.703402,0.428547,0.428249,0.427241,0.426157
2,0.581600,2.284380,0.493917,0.489437,0.492964,0.491156
3,0.375400,2.556563,0.495499,0.494268,0.496036,0.496448


('./tiny-llama-squad-finetuned/tokenizer_config.json',
 './tiny-llama-squad-finetuned/special_tokens_map.json',
 './tiny-llama-squad-finetuned/tokenizer.model',
 './tiny-llama-squad-finetuned/added_tokens.json',
 './tiny-llama-squad-finetuned/tokenizer.json')

In [22]:
output_dir = "./tiny-llama-squad-finetuned-final"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('./tiny-llama-squad-finetuned-final/tokenizer_config.json',
 './tiny-llama-squad-finetuned-final/special_tokens_map.json',
 './tiny-llama-squad-finetuned-final/tokenizer.model',
 './tiny-llama-squad-finetuned-final/added_tokens.json',
 './tiny-llama-squad-finetuned-final/tokenizer.json')

In [23]:
!zip -r tiny-llama-final.zip tiny-llama-squad-finetuned-final/

  adding: tiny-llama-squad-finetuned-final/ (stored 0%)
  adding: tiny-llama-squad-finetuned-final/tokenizer.model (deflated 55%)
  adding: tiny-llama-squad-finetuned-final/adapter_model.safetensors (deflated 7%)
  adding: tiny-llama-squad-finetuned-final/tokenizer_config.json (deflated 68%)
  adding: tiny-llama-squad-finetuned-final/adapter_config.json (deflated 56%)
  adding: tiny-llama-squad-finetuned-final/tokenizer.json (deflated 85%)
  adding: tiny-llama-squad-finetuned-final/README.md (deflated 66%)
  adding: tiny-llama-squad-finetuned-final/special_tokens_map.json (deflated 79%)
  adding: tiny-llama-squad-finetuned-final/training_args.bin (deflated 52%)


In [27]:
# Load fine-tuned model
output_dir = "./tiny-llama-squad-finetuned-final/"
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained(output_dir)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(output_dir)
fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    from_tf=bool(".ckpt" in output_dir),
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [28]:
fine_tuned_model_pipeline = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=fine_tuned_tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.3,
    return_full_text=False,
    pad_token_id=fine_tuned_tokenizer.eos_token_id
)

Device set to use cuda:0


In [29]:
fine_tuned_llm = HuggingFacePipeline(pipeline=fine_tuned_model_pipeline)

# 3 - Experiments

In [30]:
# import the product csv
file = 'drive/MyDrive/validation.csv'
loader = CSVLoader(file_path=file)

In [31]:
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

<ipython-input-31-6c19354b71ea>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
# create the vector database
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [33]:
docs = loader.load()
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)
retriever = db.as_retriever()

In [56]:
# Prompt Templates

# Zero-shot prompt template
zero_shot_template = PromptTemplate(
    # input_key="context",
    template="""Below is a context and a question about it. Answer the question based on the given context.

    {context}

    Answer:"""
)

# Chain-of-thought prompt template
cot_template = PromptTemplate(
    # input_key="message",
    template="""Let's think step by step. Below is a context and a question about it. Answer the question based on the given context.

    {context}

    Answer:"""
)


In [57]:
# QA Chains

# Zero-shot RetrievalQA chain
qa_zero_shot = RetrievalQA.from_chain_type(
    llm=fine_tuned_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": zero_shot_template},
    return_source_documents=False
)

# Chain-of-thought RetrievalQA chain
qa_cot = RetrievalQA.from_chain_type(
    llm=fine_tuned_llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": cot_template},
    return_source_documents=False
)


## 3.1 - Zero-Shot Prompting + RAG

In [58]:
def respond_zero_shot(message, chat_history):
    # Get response from conversation chain
    response = qa_zero_shot.run(message)

    # Update chat history
    chat_history.append((message, response))

    return "", chat_history

In [59]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond_zero_shot, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond_zero_shot, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

<ipython-input-59-603c9a006e54>:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=240) #just to fit the notebook


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f1b9abc1e022ca18b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 3.2 - COT Prompting + RAG

In [60]:
def respond_cot(message, chat_history):
    # Get response from conversation chain
    response = qa_cot.run(message)

    # Update chat history
    chat_history.append((message, response))

    return "", chat_history

In [61]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond_cot, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond_cot, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

<ipython-input-61-ccfd1465f557>:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=240) #just to fit the notebook


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3412d991ba937cf013.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 3.3 - Fine-Tuned LLM (No RAG)

In [67]:
prompt_template = PromptTemplate(
    template="""Below is a context and a question about it. Answer the question based on the given context.

    {context}

    Answer:""",
    input_variables=["context"]
)

llm_chain = LLMChain(
    llm=fine_tuned_llm,
    prompt=prompt_template,
    verbose=True,
    # memory=memory
)

In [68]:
def respond_cot(message, chat_history):
    # Get response from conversation chain
    response = llm_chain.run(message)

    # Update chat history
    chat_history.append((message, response))

    return "", chat_history

In [69]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond_cot, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond_cot, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit

gr.close_all()
demo.launch()

<ipython-input-69-ccfd1465f557>:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=240) #just to fit the notebook


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f31d16d4982c524db1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
